<a href="https://colab.research.google.com/github/Homura-san/pyspark/blob/main/Pyspark_ElasticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalando o Pyspark**

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [5]:
# instalando a findspark
!pip install -q findspark

In [6]:
# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

In [7]:
# importando o pacote necessário para iniciar uma seção Spark
from pyspark.sql import SparkSession

# iniciando o spark context
sc = SparkSession.builder.master('local[*]').getOrCreate()

# Verificando se a sessão foi criada
sc

# **Instalando o ElasticSearch**

In [8]:
#!pip install tensorflow-io
!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.9 MB/s eta 0:00:00


In [9]:
import os
import time
from sklearn.model_selection import train_test_split
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
#import tensorflow as tf
#from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing
#import tensorflow_io as tfio

### Validando imports do tf e tfio

In [ ]:
#print("tensorflow-io version: {}".format(tfio.__version__))
#print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.32.0
tensorflow version: 2.12.0


## Baixando e instalando a instância do Elasticsearch

In [10]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.15.0-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.15.0-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-7.15.0-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.15.0/
shasum -a 512 -c elasticsearch-7.15.0-linux-x86_64.tar.gz.sha512

elasticsearch-7.15.0-linux-x86_64.tar.gz: OK


In [11]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.15.0/bin/elasticsearch

In [12]:
# Sleep for few seconds to let the instance start.
time.sleep(20)

In [17]:
%%bash

ps -ef | grep elasticsearch

root        1649    1647  0 13:25 ?        00:00:00 sudo -H -u daemon elasticsearch-7.15.0/bin/elasticsearch
daemon      1650    1649 99 13:25 ?        00:00:50 /content/elasticsearch-7.15.0/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=ALL-UNNAMED -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-11856754181015760773 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Xms6491m -X

In [19]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "b50329bcae77",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "qZQTwtf6QjGJ-PPfrvmn5g",
  "version" : {
    "number" : "7.15.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "79d65f6e357953a5b3cbcc5e2c7c21073d89aa29",
    "build_date" : "2021-09-16T03:05:29.143308416Z",
    "build_snapshot" : false,
    "lucene_version" : "8.9.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


# **Importando Arquivo para teste**

#### * Via Pyspark

In [ ]:
# Fazendo download do arquivo
!wget --verbose --show-progress --no-check-certificate https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv

--2023-05-19 17:32:45--  https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6216 (6.1K) [text/plain]
Saving to: ‘receita_federal_arrecadacao_por_UF_2020.csv’

receita_federal_arr 100%[===================>]   6.07K  --.-KB/s    in 0s      

2023-05-19 17:32:45 (51.6 MB/s) - ‘receita_federal_arrecadacao_por_UF_2020.csv’ saved [6216/6216]



In [ ]:
# carregando um conjunto de dados que baixamos da internet
receitafederal = sc.read.csv(
    path = "/content/receita_federal_arrecadacao_por_UF_2020.csv", 
    inferSchema = True, 
    header = True,
    sep = ';', 
    encoding = "UTF-8")

In [ ]:
# Verificando o tipo de objeto criado
type(receitafederal)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Espiando o dataset
receitafederal.show()

+---+------------+----+------------------------+------------------------+-------------+---------------------------+-------------+--------------+-------------------------------+-------------------------------+-------------------------+--------------+-----------------------------+--------------+-----------------+------------------------------------------------------+-----------------------------+
| uf|      regiao| ano|imposto_sobre_importacao|imposto_sobre_exportacao|    ipi_total|imposto_sobre_a_renda_total|         irpf|          irpj|imposto_s_renda_retido_na_fonte|imposto_s_operacoes_financeiras|imposto_territorial_rural|        cofins|contribuicao_para_o_pis_pasep|          csll|cide_combustiveis|cpsss_contrib_p_o_plano_de_segurid_social_serv_publico|outras_receitas_administradas|
+---+------------+----+------------------------+------------------------+-------------+---------------------------+-------------+--------------+-------------------------------+----------------------------

In [ ]:
# Verificando o schema() deste sparkdataframe
receitafederal.printSchema()

root
 |-- uf: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- imposto_sobre_importacao: string (nullable = true)
 |-- imposto_sobre_exportacao: string (nullable = true)
 |-- ipi_total: string (nullable = true)
 |-- imposto_sobre_a_renda_total: string (nullable = true)
 |-- irpf: string (nullable = true)
 |-- irpj: string (nullable = true)
 |-- imposto_s_renda_retido_na_fonte: string (nullable = true)
 |-- imposto_s_operacoes_financeiras: string (nullable = true)
 |-- imposto_territorial_rural: string (nullable = true)
 |-- cofins: string (nullable = true)
 |-- contribuicao_para_o_pis_pasep: string (nullable = true)
 |-- csll: string (nullable = true)
 |-- cide_combustiveis: string (nullable = true)
 |-- cpsss_contrib_p_o_plano_de_segurid_social_serv_publico: string (nullable = true)
 |-- outras_receitas_administradas: string (nullable = true)



In [ ]:
# importando os métodos com funções para transformações de variáveis
from pyspark.sql.functions import *


In [ ]:
# Transformando o atributo irpf em numerica
receitafederal = receitafederal.withColumn(
    colName = 'irpf', 
    col = regexp_replace('irpf',',','.').cast('float')
    )

# Inspecionando o resultado
receitafederal.select('irpf').printSchema()

root
 |-- irpf: float (nullable = true)



In [ ]:
# Verificando o total do irpf por Região do Brasil
receitafederal.groupBy('regiao').sum('irpf').orderBy('regiao').show()

+------------+--------------+
|      regiao|     sum(irpf)|
+------------+--------------+
|Centro-Oeste| 3.354157696E9|
|    Nordeste| 4.303029696E9|
|       Norte| 1.404179308E9|
|     Sudeste|2.496098528E10|
|         Sul| 7.380957184E9|
|       Total|4.140331008E10|
+------------+--------------+



### * Via ElasticSearch

In [ ]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'
tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
pf_df = pd.read_csv(csv_file)

In [ ]:
pf_df.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [ ]:
# Number of datapoints and columns
len(pf_df), len(pf_df.columns)

(11537, 14)

# Usando arquivo prórpio

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- Via pyspark

In [ ]:
inflacao = sc.read.csv(
    path = "/content/drive/My Drive/IPCA.csv", 
    inferSchema = True, 
    header = True,
    sep = ',', 
    encoding = "UTF-8")

In [ ]:
# Verificando o tipo de objeto criado
type(inflacao)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Olhando o dataset
inflacao.show()

+-----------------+-----------+-----------------------+-------------------+
|MES DE REFERENCIA|IPCA NO MES|IPCA ACUMULADO 12 MESES|IPCA ACUMULADO 2022|
+-----------------+-----------+-----------------------+-------------------+
|              Jan|      0.54%|                 10.38%|              0.54%|
|              Fev|      1.01%|                 10.54%|              1.56%|
|              Mar|      1.62%|                 11.30%|              3.20%|
|              Abr|      1.06%|                 12.13%|              4.29%|
|              Mai|      0.47%|                 11.73%|              4.78%|
|              Jun|      0.67%|                 11.89%|              5.49%|
|              Jul|     -0.68%|                 10.07%|              4.77%|
|              Ago|     -0.36%|                  8.73%|              4.39%|
|              Set|     -0.29%|                  7.17%|              4.09%|
|              Out|      0.59%|                  6.47%|              4.70%|
|           

In [ ]:
inflacao.printSchema()

root
 |-- MES DE REFERENCIA: string (nullable = true)
 |-- IPCA NO MES: string (nullable = true)
 |-- IPCA ACUMULADO 12 MESES: string (nullable = true)
 |-- IPCA ACUMULADO 2022: string (nullable = true)



- Via ElasticSearch

In [21]:
import pandas as pd 
ipca2022 = pd.read_csv('/content/drive/My Drive/IPCA.csv', sep=",")

In [22]:
ipca2022.head()

,MES DE REFERENCIA,IPCA NO MES,IPCA ACUMULADO 12 MESES,IPCA ACUMULADO 2022
0,Jan,0.54%,10.38%,0.54%
1,Fev,1.01%,10.54%,1.56%
2,Mar,1.62%,11.30%,3.20%
3,Abr,1.06%,12.13%,4.29%
4,Mai,0.47%,11.73%,4.78%


In [23]:
ES_NODES = "http://localhost:9200"

def prepare_es_data(index, doc_type, df):
  records = df.to_dict(orient="records")
  es_data = []
  for idx, record in enumerate(records):
    meta_dict = {
          "index": {
              "_index": index, 
              "_type": doc_type, 
              "_id": idx
          }
      }
    es_data.append(meta_dict)
    es_data.append(record)

  return es_data

def index_es_data(index, es_data):
  es_client = Elasticsearch(hosts = [ES_NODES])
  if es_client.indices.exists(index = index):
      print("deleting the '{}' index.".format(index))
      res = es_client.indices.delete(index=index)
      print("Response from server: {}".format(res))

  print("creating the '{}' index.".format(index))
  res = es_client.indices.create(index=index)
  print("Response from server: {}".format(res))

  print("bulk index the data")
  res = es_client.bulk(index=index, body=es_data, refresh = True)
  print("Errors: {}, Num of records indexed: {}".format(res["errors"], len(res["items"])))

In [24]:
ipca2022_es_data = prepare_es_data(index="ipca", doc_type="mes", df=ipca2022)
index_es_data(index="ipca", es_data=ipca2022_es_data)
time.sleep(3)

<ipython-input-23-16b4e0734aae>:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  if es_client.indices.exists(index = index):


creating the 'ipca' index.


<ipython-input-23-16b4e0734aae>:27: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  res = es_client.indices.create(index=index)
<ipython-input-23-16b4e0734aae>:31: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'operations' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es_client.bulk(index=index, body=es_data, refresh = True)


Response from server: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'ipca'}
bulk index the data


<ipython-input-23-16b4e0734aae>:31: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  res = es_client.bulk(index=index, body=es_data, refresh = True)
<ipython-input-23-16b4e0734aae>:31: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  res = es_client.bulk(index=index, body=es_data, refresh = True)


Errors: False, Num of records indexed: 12


In [28]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200, "scheme": "http"}])
# Check if python is connected to elasticsearch
es.ping()

<ipython-input-28-9aa3f0dac12f>:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  es.ping()


True

In [29]:
# Get 10 sample of data
query = es.search(
    index="ipca",
    body={
      "size":10,
      "query": {
        "match_all":{}
      }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

<ipython-input-29-83fc3410e5e9>:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  query = es.search(
<ipython-input-29-83fc3410e5e9>:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  query = es.search(


,_index,_type,_id,_score,_source.MES DE REFERENCIA,_source.IPCA NO MES,_source.IPCA ACUMULADO 12 MESES,_source.IPCA ACUMULADO 2022
0,ipca,mes,0,1.0,Jan,0.54%,10.38%,0.54%
1,ipca,mes,1,1.0,Fev,1.01%,10.54%,1.56%
2,ipca,mes,2,1.0,Mar,1.62%,11.30%,3.20%
3,ipca,mes,3,1.0,Abr,1.06%,12.13%,4.29%
4,ipca,mes,4,1.0,Mai,0.47%,11.73%,4.78%
5,ipca,mes,5,1.0,Jun,0.67%,11.89%,5.49%
6,ipca,mes,6,1.0,Jul,-0.68%,10.07%,4.77%
7,ipca,mes,7,1.0,Ago,-0.36%,8.73%,4.39%
8,ipca,mes,8,1.0,Set,-0.29%,7.17%,4.09%
9,ipca,mes,9,1.0,Out,0.59%,6.47%,4.70%
